## Gpy with Emukit wrapper

In [ ]:
import GPy
from emukit.model_wrappers import GPyModelWrapper


input_dim = len(X[0]) # number of process variables

# Create the kernel with the best hyperparameters
kernel = GPy.kern.Matern52( #RBF/Matern, use ARD
    input_dim=input_dim,
    ARD=True
)

# Create the GP classifier with the best hyperparameters
model_gpy = GPy.models.GPRegression(X, Y, kernel=kernel)

# Optimize using Gpy
kernel.lengthscale.constrain_bounded(0.1, 1) #upper bound set to 1
kernel.variance.constrain_bounded(0.1, 1000.0)
model_gpy.randomize()
model_gpy.optimize_restarts(num_restarts=100,verbose =False, messages=False)
model_gpy.optimize()

emukit_model = GPyModelWrapper(model_gpy)

## Sklearn with Emukit Wrapper

In [ ]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern
from emukit.core.interfaces import IModel

class SklearnGPModel(IModel):
    def __init__(self, sklearn_model):
        self.model = sklearn_model

    def predict(self, X):
        mean, std = self.model.predict(X, return_std=True)
        return mean[:, None], np.square(std)[:, None]

    def set_data(self, X: np.ndarray, Y: np.ndarray) -> None:
        self.model.fit(X, Y)

    def optimize(self, verbose: bool = False) -> None:
        # There is no separate optimization routine for sklearn models
        pass

    @property
    def X(self) -> np.ndarray:
        return self.model.X_train_

    @property
    def Y(self) -> np.ndarray:
        return self.model.y_train_


X, Y = [x_init, y_init.ravel()] # Flatten output dimensions to (N,)

input_dim = len(X[0]) # number of process variables

# kernel = 1.0 * RBF(length_scale=np.ones(input_dim),
#                         length_scale_bounds=(1e-2, 1e5))

kernel = 1.0 * Matern(nu=2.5, #matern 52
                      length_scale=np.ones(input_dim),
                        length_scale_bounds=(1e-2, 1e5))

sklearn_gp = GaussianProcessRegressor(kernel=kernel, 
                                  random_state=42,
                                  n_restarts_optimizer=100,
                                  n_jobs=-1
                                  )
sklearn_gp.fit(X,Y)

sklearn_gp.kernel_

sklearn_gp.predict(new_x) # predict at new x location with trained GP

emukit_model = SklearnGPModel(sklearn_gp)


## SKlearn with Bayes Opt optimizer

## https://github.com/bayesian-optimization/BayesianOptimization

In [ ]:
from bayes_opt import BayesianOptimization,
from sklearn.gaussian_process.kernels import Matern

# Bounded region of parameter space
parameter_space = {"Precursor Concentration":
                        (0 - 1 / (precursor_num - 1) / 2,
                        1 + 1 / (precursor_num - 1) / 2), 
                   "SuperTemp": 
                       (0 - 1 / (super_temp_num - 1) / 2,
                        1 + 1 / (super_temp_num - 1) / 2),
                   "Speed": 
                       (0 - 1 / (speed_num - 1) / 2, 
                        1 + 1 / (speed_num - 1) / 2),
                   "Distance": 
                       (0 - 1 / (dist_num - 1) / 2,
                        1 + 1 / (dist_num - 1) / 2),
                   "DMSO": 
                       (0 - 1 / (dmso_num - 1) / 2,
                        1 + 1 / (dmso_num - 1) / 2),
                   "SubTemp": 
                       (0 - 1 / (sub_temp_num - 1) / 2,
                        1 + 1 / (sub_temp_num - 1) / 2),
                   "AnnealTime": 
                       (0 - 1 / (anneal_num - 1) / 2,
                        1 + 1 / (anneal_num - 1) / 2)
                   }

X, Y = [x_init, y_init.ravel()] # Flatten output dimensions to (N,)

input_dim = len(X[0])  # Get number of variables in space

# Create Bayes Opt Optimizer Instance
optimizer = BayesianOptimization(f=None, #real-world function
                                 pbounds=parameter_space, #parameter space above
                                 random_state=42, #to make it reproducible
                                 allow_duplicate_points=True) #dont stop optimizer when duplicate points are suggested


# replace optimizer kernel to nonisotropic Matern 5/2. Needed for multidimensional Bayesian Optimization.
kernel = 1.0 * Matern(nu=2.5,
            length_scale=np.ones(len(parameter_space)), # default = 1.0
            length_scale_bounds=(1e-1, 1e4))

# set params of underlying GP
optimizer.set_gp_params(kernel=kernel, 
                        n_restarts_optimizer=50,
                         # alpha=1e-3
                        )

# get dataframe/dict of current inputs/output(s)
df_X = pd.DataFrame(X, columns=parameter_space.keys())
current_observations = df_X.to_dict('records')

# Set X data of model
for i, observer_dict in enumerate(current_observations):
    # print(observer_dict)
    # print(Y[i])
    optimizer.register( # get initial data
        params=observer_dict,
        target=Y[i],
    )

#fit model 
optimizer._gp.fit(X, Y)

#predict
y_pred = optimizer._gp.predict(X)
